In [ ]:
# Ghi âm tín hiệu nhận được qua GUI recorder và lưu dưới tên file rec.wav
import sys
import wave

import pyaudio as pa
from PyQt6.QtCore import QThread, pyqtSignal, pyqtSlot
from PyQt6.QtWidgets import *


class RecordingThread(QThread):
    stopped = False
    sig_started = pyqtSignal()
    sig_stopped = pyqtSignal()

    def __init__(self, target_file):
        self.target_file = target_file
        super().__init__()

    def run(self) -> None:
        audio = pa.PyAudio()
        frames = []
        stream = audio.open(format=pa.paInt16, channels=1, rate=44100, input=True, frames_per_buffer=1024)
        self.stopped = False
        self.sig_started.emit()

        while not self.stopped:
            data = stream.read(1024)
            frames.append(data)

        stream.close()

        self.sig_stopped.emit()

        wf = wave.open(self.target_file, 'wb')
        wf.setnchannels(1)
        wf.setsampwidth(audio.get_sample_size(pa.paInt16))
        wf.setframerate(44100)
        wf.writeframes(b''.join(frames))
        wf.close()

    @pyqtSlot()
    def stop(self):
        self.stopped = True


class Window(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("Rec Audio")
        # Tạo luồng ghi âm 
        self.recording_thread = RecordingThread(target_file='rec.wav')
        self.recording_thread.sig_started.connect(self.recording_started)
        self.recording_thread.sig_stopped.connect(self.recording_stopped)

        vbox = QVBoxLayout()

        self.labelRec = QLabel('')
        self.labelRec.setFixedSize(260, 30)

        hbox = QHBoxLayout()
        self.recbtn = QPushButton('▶ RECORD')
        self.recbtn.setFixedSize(180, 60)
        
        """Nhấp vào nút ghi âm (recbtn), kích hoạt slot “start” của đối tượng QThread"""
        self.recbtn.clicked.connect(self.recording_thread.start)

        self.stopbtn = QPushButton('▪ STOP')
        self.stopbtn.setDisabled(True)
        self.stopbtn.setFixedSize(80, 60)

        """Nhấp vào nút dừng (stopbtn), kích hoạt slot “stop” của đối tượng QThread"""
        self.stopbtn.clicked.connect(self.recording_thread.stop)
        hbox.addWidget(self.recbtn)
        hbox.addWidget(self.stopbtn)

        vbox.addWidget(self.labelRec)
        vbox.addLayout(hbox)

        self.setLayout(vbox)

    @pyqtSlot()
    def recording_started(self):
        """Slot được gọi khi bắt đầu ghi âm"""
        self.labelRec.setText('◉ RECORDING...')
        self.stopbtn.setDisabled(False)
        self.recbtn.setDisabled(True)

    @pyqtSlot()
    def recording_stopped(self):
        """Slot này được gọi khi dừng ghi âm"""
        self.labelRec.setText('recording stopped')
        self.recbtn.setDisabled(False)
        self.stopbtn.setDisabled(True)


if __name__ == "__main__":
    app = QApplication(sys.argv)
    win = Window()
    win.show()
    app.exec()

In [ ]:
# Lọc bỏ các tần số khác tần số sóng mang tín hiệu
import scipy.io.wavfile as wavfile
from scipy import signal
import warnings

# Đọc file rec.wav
warnings.filterwarnings('ignore', category=wavfile.WavFileWarning)
sample_rate, data = wavfile.read('rec.wav')

# Tạo bộ lọc thông dải Butterworth cho khoảng tần số 5000Hz
nyquist_rate = sample_rate / 2.0
low = 4900 / nyquist_rate
high = 5100 / nyquist_rate
b1, a1 = signal.butter(5, [low, high], btype='band')

# Tạo bộ lọc thông dải Butterworth cho tần số 15000Hz
low = 14900 / nyquist_rate
high = 15100 / nyquist_rate
b2, a2 = signal.butter(5, [low, high], btype='band')

# Áp dụng bộ lọc thứ nhất
filtered_data1 = signal.filtfilt(b1, a1, data)

# Áp dụng bộ lọc thứ hai
filtered_data2 = signal.filtfilt(b2, a2, data)

# Tổng hợp 2 dữ liệu âm thanh đã lọc
filtered_data = filtered_data1 + filtered_data2

# Lưu file wav đã lọc dưới tên filtered_rec.wav
wavfile.write('filtered_rec.wav', sample_rate, filtered_data.astype(data.dtype))

In [ ]:
# Thống kê các tần số của file đã lọc
import matplotlib.pyplot as plt
import numpy as np
from scipy.io import wavfile
from scipy import signal

# Đọc file wav đã lọc
sample_rate, data = wavfile.read('filtered_rec.wav')

# Calculate the power spectral density
frequencies, power_spectral_density = signal.periodogram(data, sample_rate)

# Vẽ đồ thị PSD
plt.semilogy(frequencies, power_spectral_density)
plt.xlabel('Frequency [Hz]')
plt.ylabel('Power spectral density [V**2/Hz]')
plt.show()

In [ ]:
# Giải điều chế âm thanh nhận được
import warnings
import numpy as np
from scipy import signal
from scipy.io.wavfile import read

def decode(audio_signal, sample_rate=44100, baud_rate=50):
    samples_per_bit = sample_rate // baud_rate
    num_bits = len(audio_signal) // samples_per_bit
    binary_string = ''
    for i in range(num_bits):
        bit_signal = audio_signal[i*samples_per_bit:(i+1)*samples_per_bit]
        f, Pxx = signal.periodogram(bit_signal, sample_rate)
        
        if f[np.argmax(Pxx)] > 6000:
            binary_string += '1'
        else:
            binary_string += '0'
    return binary_string


with warnings.catch_warnings():
    warnings.simplefilter('ignore')
    sample_rate, audio_signal = read('filtered_rec.wav')

# Sau khi giải mã file ghi âm ta sẽ nhận được dòng bit có các bit 0 dư thừa ở đầu và cuối tín hiệu
decoded_binary_string = decode(audio_signal, sample_rate)
print(decoded_binary_string)

In [ ]:
# Xoá bỏ chuỗi tiền tố, hậu tố, các bit 0 dư thừa
def remove_extra_bits(binary_string, preamble='010101', postamble='101010'):
    start_index = binary_string.find(preamble) + len(preamble)
    end_index = binary_string.rfind(postamble)
    if start_index == -1 or end_index == -1:
        print('Tín hiệu nhận bị lỗi')
        return
    return binary_string[start_index:end_index]

# Định nghĩa chuỗi tiền tố và hậu tố
preamble = '010101'
postamble = '101010'

# Giải mã và in kết quả
binary_string = remove_extra_bits(decoded_binary_string, preamble, postamble)
if binary_string:
    print(binary_string)

In [ ]:
# Giải mã RLE về văn bản ban đầu

def run_length_decoding(binary_string):
    if not binary_string:
        print('Tín hiệu nhận bị lỗi')
        return
    decoded_text = ""
    n = int(len(binary_string)/8)
    encoded_text = ''.join(chr(int(binary_string[i*8:i*8+8], 2)) for i in range(n))
    i = 0
    while i < len(encoded_text):
        count = int(encoded_text[i])
        char = encoded_text[i + 1]
        decoded_text += char * count
        i += 2
    return decoded_text

decoded_text = run_length_decoding(binary_string)

# In ra các văn bản nhận được
if decoded_text:
    print(decoded_text)
file_name = 'decoded_text.txt'
with open(file_name, 'w') as f:
    f.write(decoded_text)